In [1]:
import os,sys

os.getcwd()

'/home/fin/tmp/fis-ski'

In [7]:
import unicodecsv

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))

all_ergebnisse = []
errorcount=0
errors = []

import re
numbersonly = re.compile('=([0-9]+)')

all_races = {}

with open('fin_output.csv') as ergebnisse, open('subpages.csv') as subpages:
    er = read_lines(unicodecsv.reader(ergebnisse))
     
    sr = unicodecsv.reader(subpages)
    
    all_races = dict((numbersonly.findall(x['url'])[0],x,) for x in read_lines(sr))
    
    for e in er:
        try:
            e['race'] = all_races[numbersonly.findall(e['source_url'])[0]]
            all_ergebnisse.append(e)
        except:
            errorcount+=1
            errors.append(e)

In [13]:
all_ergebnisse[0]

{u'Bib': u'',
 u'Diff.': u'&nbsp',
 u'FIS Code': u'55198',
 u'FIS Points': u'-4.00',
 u'Name': u'DORFMEISTER Michaela',
 u'Nation': u'AUT',
 u'Rank': u'1',
 u'Run 1': u'1:07.95',
 u'Run 2': u'1:11.93',
 u'Total Time': u'2:19.88',
 u'Year': u'1973',
 'race': {u'category': u'WC',
  u'codex': u'5127',
  u'date': u'09.12.2000',
  u'disziplin': u'Giant Slalom',
  u'gender': u'L',
  u'kommentar': u'Val d Is\xe8re06.12',
  u'land': u'ITA',
  u'ort': u'Sestriere',
  u'url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'},
 u'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'}

In [41]:
def sortgroup(xs, keyfun):
    import itertools
    return itertools.groupby(sorted(xs,key=keyfun),key=keyfun)

by_race = dict([(x,
                 list(sorted(
                    [z for z in y if z['Rank'] and z['Rank'].isdigit()]
                    , key=lambda e: int(e['Rank'],10))),
                )
                for (x,y,) in sortgroup(all_ergebnisse, lambda x: x['source_url'])
               ])

In [40]:
''.isdigit()

False

In [70]:
pointreplaced = 0

for raceurl,finishers in by_race.iteritems():

    last = None
    for f in finishers:
        if f['Total Time'].strip():
            if last:
                if ':' in last['Total Time'] and not ':' in f['Total Time']:
                    pointreplaced+=1
                    f['Total Time'] = f['Total Time'].replace('.',':',1)
            if ':' in f['Total Time']:
                minutes,rest = f['Total Time'].split(':',1)
            else:
                minutes = '0'
                rest = f['Total Time']
            
            seconds,nths = re.split('[\.,]',rest)

            minutes = int(minutes,10)
            seconds = int(seconds,10)
            nths_i = int(nths,10)
            nths_f = nths_i/(pow(10.0,len(nths)))
            finalseconds = minutes*60 + seconds + nths_f

            f['mytime'] = finalseconds

            f['diff'] = f['mytime']-finishers[0]['mytime']
            if last and hasattr(last,'mytime'):
                if f['diff']<last['diff']:
                    print 'time travel?'
                    print f['Total Time'],f['mytime'],last['Total Time'],last['mytime'],f['source_url']
                    print seconds
                    print nths, len(nths), nths_i, nths_f
    
        last = f

In [74]:
import itertools
all_diffs = [x for x in itertools.chain(*by_race.values()) if hasattr(x,'diff')]

In [ ]:
by_race.values()

In [49]:
finishers

[{u'Bib': u'',
  u'Diff.': u'&nbsp',
  u'FIS Code': u'-10868',
  u'FIS Points': u'?????',
  u'Name': u'MAHRE Steve',
  u'Nation': u'USA',
  u'Rank': u'1',
  u'Run 1': u'',
  u'Run 2': u'',
  u'Total Time': u'1:20.07',
  u'Year': u'1957',
  'diff': 0.0,
  'mytime': 80.35,
  'race': {u'category': u'WC',
   u'codex': u'1326',
   u'date': u'11.01.1981',
   u'disziplin': u'Slalom',
   u'gender': u'M',
   u'kommentar': u'',
   u'land': u'GER',
   u'ort': u'Garmisch-Partenkirchen',
   u'url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=8949'},
  u'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=8949'},
 {u'Bib': u'',
  u'Diff.': u'+0.50',
  u'FIS Code': u'-10745',
  u'FIS Points': u'?????',
  u'Name': u'POPANGELOV Peter',
  u'Nation': u'BUL',
  u'Rank': u'2',
  u'Run 1': u'',
  u'Run 2': u'',
  u'Total Time': u'1:20.57',
  u'Year': u'1959',
  'diff': 2.5,
  'mytime': 82.85,
  'race': {u'category': u'WC',
   u'codex': u'1326',
   u'date': u'11.0

In [44]:
pointreplaced

1

In [39]:
print f['Total Time']
print f['source_url']

import collections

#collections.Counter([x['Total Time'] for x in all_ergebnisse if '.' not in x['Total Time']])
[x for x in all_ergebnisse if x['Total Time']=='0'][0]

3.59.96
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=65970


{u'Bib': u'',
 u'Diff.': u'',
 u'FIS Code': u'685003',
 u'FIS Points': u'',
 u'Name': u'AKHMETELI Sofia',
 u'Nation': u'GEO',
 u'Rank': u'',
 u'Run 1': u'',
 u'Run 2': u'',
 u'Total Time': u'0',
 u'Year': u'1981',
 'race': {u'category': u'WC',
  u'codex': u'5127',
  u'date': u'09.12.2000',
  u'disziplin': u'Giant Slalom',
  u'gender': u'L',
  u'kommentar': u'Val d Is\xe8re06.12',
  u'land': u'ITA',
  u'ort': u'Sestriere',
  u'url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'},
 u'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'}

In [99]:
i=0
for raceurl, finishers in by_race.iteritems():
    x = fispoints.findall(finishers[0]['time'])
    y = fispoints.findall(finishers[1]['time'])
    z = fispoints.findall(finishers[2]['time'])
    by_truth = [b for b in (x,y,z,) if b]
    if(len(by_truth)!=3 and len(by_truth)!=0):
        i+=1
        print x,y,z
        print raceurl

[] [u' 4.03'] [u' 8.76']
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=4208
[u' 0.00'] [] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=47783
[u' 0.00'] [u' 8.38'] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=47786
[u' 0.00'] [u' 8.64'] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=59289
[u' 0.00'] [u' 6.57'] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=59280
[u' 0.00'] [u' 9.46'] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=47955
[u' 0.00'] [u' 8.61'] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=47957
[] [] [u' 3.19']
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13415
[] [] [u' 3.74']
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13414
[] [] [u' 0.29']
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=3854
[u' 0.00'] [] []
http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=79139
[u' 0.00'] [] []
ht

IndexError: list index out of range

In [95]:
finishers[0]['raceurl']

u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=4208'

In [61]:
races_with_time = set([x['raceurl'] for x in all_ergebnisse if x['rang'].strip() and x['time'].strip()])
races_witout_time = set([x['raceurl'] for x in all_ergebnisse if x['rang'].strip() and not x['time'].strip()])

In [63]:
len(races_with_time & races_witout_time)

45